In [ ]:
import os
from diarize import 
from transcriptTools import datesDict, titlesDict, loadSpeakerLabels, getTranscriptFiles, srt_to_transcript

ROOT = os.getcwd()

labels = loadSpeakerLabels("speaker_labels.csv")
rotl_titles = titlesDict("rotl_titles.txt")
rotl_dates = datesDict("rotl_dates.txt")
roadwork_titles = titlesDict("roadwork_titles.txt")
roadwork_dates = datesDict("roadwork_dates.txt")


def getTitle(showname, episode):
    if showname == "rotl":
        return rotl_titles[episode]
    else:
        return roadwork_titles[episode]


def getDate(showname, episode):
    if showname == "rotl":
        return rotl_dates[episode]
    else:
        return roadwork_dates[episode]


def getLabel(showname, episode, speaker):
    return labels[showname][episode][speaker]


def getDuration(start, end):
    if start > end:
        return 0.000
    else:
        return round(end - start, 3)


transcriptDir = os.path.join(ROOT, "transcripts")
files = getTranscriptFiles(transcriptDir)

In [ ]:
import sqlite3

dbFile = "transcripts.db"

conn = sqlite3.connect(dbFile)

c = conn.cursor()

c.execute(
    """CREATE TABLE lines (
        id integer primary key,
        filename text,
        showname text,
        episode text,
        title text,
        date text,
        idx integer,
        start real,
        end real,
        duration real,
        speaker text,
        label text,
        speech text
        )"""
)

conn.commit()

In [ ]:
for file in files:
    filepath, showname, filename = file
    episode = filename.split("_-_")[0]
    title = getTitle(showname, episode)
    date = getDate(showname, episode)
    transcript = srt_to_transcript(filepath)
    for idx, start, end, speaker, speech in transcript:
        label = getLabel(showname, episode, speaker)
        duration = getDuration(start, end)
        c.execute(
            "INSERT INTO lines VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (
                filename,
                showname,
                episode,
                title,
                date,
                idx,
                start,
                end,
                duration,
                speaker,
                label,
                speech,
            ),
        )
    conn.commit()
conn.close()

In [ ]:
searchterm = "bed"
conn = sqlite3.connect("transcripts.db")
c = conn.cursor()

c.execute("""SELECT * FROM lines WHERE speech LIKE '%' || ? || '%'""", (searchterm,))

results = c.fetchall()

conn.commit()
conn.close()

len(results)

In [ ]:
lines = [
    speech
    for id, filename, showname, episode, title, date, idx, start, end, duration, speaker, label, speech in results
]

lines